# RNN기반 분류기

In [1]:
# 데이터 로딩
from sklearn.datasets import fetch_20newsgroups     # 20 Newsgroups 텍스트 분류 데이터셋 로드

categories = ['comp.graphics', 'sci.space', 'rec.sport.baseball']  # 사용할 뉴스그룹 카테고리 선택
newsgroups = fetch_20newsgroups(                                  # 선택한 카테고리만 데이터 로드
    subset='all',
    categories=categories
)

X = newsgroups.data                           # 뉴스 문서 텍스트 리스트
y = newsgroups.target                         # 각 문서의 클래스 인덱스 라벨

print(newsgroups.target_names)                # 클래스 인덱스 → 실제 뉴스그룹 이름
print(X[0])                                   # 첫 번째 뉴스 문서 원문 출력
print(y[0])                                   # 첫 번째 문서의 클래스 인덱스 출력


['comp.graphics', 'rec.sport.baseball', 'sci.space']
From: kjenks@gothamcity.jsc.nasa.gov
Subject: Life on Mars???
Organization: NASA/JSC/GM2, Space Shuttle Program Office 
X-Newsreader: TIN [version 1.1 PL8]
Lines: 12

I know it's only wishful thinking, with our current President,
but this is from last fall:

     "Is there life on Mars?  Maybe not now.  But there will be."
        -- Daniel S. Goldin, NASA Administrator, 24 August 1992

-- Ken Jenks, NASA/JSC/GM2, Space Shuttle Program Office
      kjenks@gothamcity.jsc.nasa.gov  (713) 483-4368

     "The man who makes no mistakes does not usually make
      anything."
        -- Edward John Phelps, American Diplomat/Lawyer (1825-1895)

2


In [2]:
# 데이터 전처리
from tensorflow.keras.preprocessing.text import Tokenizer        # 텍스트를 정수 시퀀스로 변환
from tensorflow.keras.preprocessing.sequence import pad_sequences # 시퀀스 길이 맞추기용 패딩 함수

vocab_size = 10000                                               # 사용할 최대 단어 사전 크기
max_len = 200                                                    # 모든 문장의 최대 길이

tokenizer = Tokenizer(                                          # Tokenizer 객체 생성
    num_words=vocab_size,                                       # 상위 vocab_size 단어만 사용
    oov_token='<OOV>'                                           # 미등록 단어 토큰 설정
)
tokenizer.fit_on_texts(X)                                       # 전체 텍스트 기준으로 단어 사전 생성
X_encoded = tokenizer.texts_to_sequences(X)                     # 문장을 정수 인덱스 시퀀스로 변환
X_padded = pad_sequences(                                       # 모든 시퀀스를 동일한 길이로 패딩
    X_encoded,
    maxlen=max_len
)

print(X_padded.shape)                                           # (문서 수, max_len) 형태 확인


(2954, 200)


In [3]:
# 데이터 분리 / 텐서 변환
import torch
from sklearn.model_selection import train_test_split     # 데이터 분할 함수
from torch.utils.data import TensorDataset, DataLoader   # PyTorch Dataset / DataLoader

# train / test 분리 (80% / 20%)
X_train, X_test, y_train, y_test = \
    train_test_split(
        torch.tensor(X_padded, dtype=torch.long),        # 패딩된 입력 데이터를 Long 텐서로 변환
        torch.tensor(y, dtype=torch.long),               # 라벨 데이터를 텐서로 변환
        test_size=0.2,                                   # 테스트 데이터 비율
        random_state=42                                  # 재현성 확보
    )

# train / validation 분리 (train의 20%를 validation으로 사용)
X_train, X_val, y_train, y_val = \
    train_test_split(
        X_train,
        y_train,
        test_size=0.2,                                   # 검증 데이터 비율
        random_state=42
    )

# Dataset 생성 (입력, 라벨 묶기)
train_dataset = TensorDataset(X_train, y_train)          # 학습 데이터셋
val_dataset = TensorDataset(X_val, y_val)                # 검증 데이터셋
test_dataset = TensorDataset(X_test, y_test)             # 테스트 데이터셋

# DataLoader 설정
batch_size = 64                                          # 배치 크기
train_loader = DataLoader(                               # 학습용 DataLoader
    train_dataset,
    batch_size=batch_size,
    shuffle=True                                         # 학습 데이터는 셔플
)
val_loader = DataLoader(                                 # 검증용 DataLoader
    val_dataset,
    batch_size=batch_size,
    shuffle=False
)
test_loader = DataLoader(                                # 테스트용 DataLoader
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)


In [4]:
# 모델 생성
import torch.nn as nn                              # PyTorch 신경망 모듈

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()                         # nn.Module 초기화
        # embedding → lstm → dense 구조
        self.embedding = nn.Embedding(             # 단어 인덱스를 임베딩 벡터로 변환
            vocab_size,                            # 단어 사전 크기
            embedding_dim,                         # 임베딩 벡터 차원
            padding_idx=0                          # PAD 토큰은 학습에 영향 없음
        )
        self.lstm = nn.LSTM(                       # 문맥 정보를 학습하는 LSTM
            embedding_dim,                         # 입력 차원 (임베딩 크기)
            hidden_size,                           # 은닉 상태 차원
            batch_first=True                       # (batch, seq, feature) 형태 사용
        )
        self.fc = nn.Linear(                       # 최종 분류용 선형 레이어
            hidden_size,                           # LSTM 은닉 상태 차원
            3                                      # 클래스 개수 (3개 뉴스그룹)
        )

    def forward(self, x):
        x = self.embedding(x)                      # (batch, seq) → (batch, seq, embed)
        _, (h, c) = self.lstm(x)                   # LSTM 통과 (h: 마지막 은닉 상태)
        out = self.fc(h[-1])                       # 마지막 타임스텝 은닉 상태로 분류
        return out                                 # (batch, 3) 로짓 출력


In [5]:
# 모델 학습
import torch.optim as optim                                  # 옵티마이저 모듈

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # cuda 가능하면 GPU 사용
embedding_dim = 100                                          # 임베딩 벡터 차원
hidden_size = 128                                            # LSTM 은닉 상태 차원

model = LSTMClassifier(vocab_size, embedding_dim, hidden_size).to(device)  # 모델 생성 후 GPU 이동
criterion = nn.CrossEntropyLoss()                            # 다중 클래스 분류용 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)         # Adam 옵티마이저 설정

# 학습 루프 기록용
train_losses, train_accs = [], []                            # 학습 손실/정확도 기록
val_losses, val_accs = [], []                                # 검증 손실/정확도 기록

epochs = 50                                                  # 전체 학습 에폭 수
for epoch in range(epochs):

    # 학습
    model.train()                                            # 학습 모드 전환
    train_loss, train_correct, train_total = 0, 0, 0          # 에폭 누적 변수 초기화

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 배치를 GPU로 이동
        optimizer.zero_grad()                                # 이전 gradient 초기화
        output = model(X_batch)                              # 순전파 (로짓 출력)
        loss = criterion(output, y_batch)                    # 손실 계산
        loss.backward()                                      # 역전파
        optimizer.step()                                     # 파라미터 업데이트

        train_loss += loss.detach().cpu().item()             # 배치 손실 누적
        pred = output.argmax(dim=1)                          # 가장 큰 로짓 인덱스 = 예측 클래스
        train_correct += (pred == y_batch).sum().detach().cpu().item()  # 정답 개수 누적
        train_total += len(y_batch)                          # 전체 샘플 수 누적

    train_loss /= len(train_loader)                          # 에폭 평균 학습 손실
    train_acc = train_correct / train_total                  # 에폭 학습 정확도
    train_losses.append(train_loss)                          # 학습 손실 기록
    train_accs.append(train_acc)                             # 학습 정확도 기록

    # 검증
    model.eval()                                             # 평가 모드 전환
    val_loss, val_correct, val_total = 0, 0, 0                # 검증 누적 변수 초기화

    with torch.no_grad():                                    # gradient 계산 비활성화
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 배치를 GPU로 이동
            output = model(X_batch)                           # 검증 순전파
            loss = criterion(output, y_batch)                 # 검증 손실 계산

            val_loss += loss.detach().cpu().item()            # 검증 손실 누적
            pred = output.argmax(dim=1)                       # 예측 클래스
            val_correct += (pred == y_batch).sum().detach().cpu().item()  # 정답 누적
            val_total += len(y_batch)                         # 샘플 수 누적

        val_loss /= len(val_loader)                           # 에폭 평균 검증 손실
        val_acc = val_correct / val_total                     # 에폭 검증 정확도
        val_losses.append(val_loss)                           # 검증 손실 기록
        val_accs.append(val_acc)                              # 검증 정확도 기록

    # 출력 (train_loss, val_loss)
    print(f'Epoch {epoch + 1}/{epochs}: '                     # 현재 에폭 정보 출력
          f'Train Loss {train_loss:.4f}, '
          f'Train Acc {train_acc:.4f}, '
          f'Val Loss {val_loss:.4f}, '
          f'Val Acc {val_acc:.4f}, ')


Epoch 1/50: Train Loss 1.0523, Train Acc 0.4735, Val Loss 0.9991, Val Acc 0.5137, 
Epoch 2/50: Train Loss 0.8993, Train Acc 0.6053, Val Loss 0.8621, Val Acc 0.6004, 
Epoch 3/50: Train Loss 0.6843, Train Acc 0.7196, Val Loss 0.7738, Val Acc 0.6596, 
Epoch 4/50: Train Loss 0.5067, Train Acc 0.8085, Val Loss 0.7244, Val Acc 0.6829, 
Epoch 5/50: Train Loss 0.3619, Train Acc 0.8651, Val Loss 0.6059, Val Acc 0.7590, 
Epoch 6/50: Train Loss 0.2494, Train Acc 0.9143, Val Loss 0.5693, Val Acc 0.7886, 
Epoch 7/50: Train Loss 0.1480, Train Acc 0.9519, Val Loss 0.5919, Val Acc 0.7970, 
Epoch 8/50: Train Loss 0.1380, Train Acc 0.9587, Val Loss 0.5525, Val Acc 0.7992, 
Epoch 9/50: Train Loss 0.1050, Train Acc 0.9635, Val Loss 0.5694, Val Acc 0.7886, 
Epoch 10/50: Train Loss 0.0610, Train Acc 0.9841, Val Loss 0.5322, Val Acc 0.8182, 
Epoch 11/50: Train Loss 0.0433, Train Acc 0.9899, Val Loss 0.6239, Val Acc 0.8372, 
Epoch 12/50: Train Loss 0.0270, Train Acc 0.9952, Val Loss 0.5573, Val Acc 0.8414, 
E

In [6]:
# 모델 평가
# - 정답 라벨과 모델 예측값을 사용해 classification_report 생성
from sklearn.metrics import classification_report    # 분류 성능 리포트 함수

model.eval()                                         # 모델을 평가 모드로 전환
all_preds, all_labels = [], []                       # 전체 예측값/정답 저장용 리스트

with torch.no_grad():                                # gradient 계산 비활성화
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 배치를 GPU로 이동
        output = model(X_batch)                      # 모델 예측값(로짓) 계산
        loss = criterion(output, y_batch)            # 테스트 손실 계산(로그용)
        pred = output.argmax(dim=1)                  # 가장 큰 로짓을 갖는 클래스 선택

        all_preds.extend(pred.detach().cpu().numpy())    # 예측 결과를 CPU numpy로 저장
        all_labels.extend(y_batch.detach().cpu().numpy())# 실제 라벨 저장

print(
    classification_report(                           # 클래스별 성능 지표 출력
        all_labels,                                  # 실제 라벨
        all_preds,                                   # 예측 라벨
        target_names=newsgroups.target_names          # 클래스 이름 매핑
    )
)


                    precision    recall  f1-score   support

     comp.graphics       0.82      0.85      0.83       202
rec.sport.baseball       0.86      0.92      0.89       202
         sci.space       0.85      0.77      0.81       187

          accuracy                           0.85       591
         macro avg       0.85      0.84      0.84       591
      weighted avg       0.85      0.85      0.85       591



## 사전학습된 임베딩 적용하기

In [7]:
%pip install gensim -q

Note: you may need to restart the kernel to use updated packages.


In [8]:
from gensim.models import FastText                 # FastText 단어 임베딩 모델 클래스

fasttext_model = FastText.load('ted_en_fasttext.model')  # 사전 학습된 FastText 모델 로드
print(fasttext_model.vector_size)                  # 각 단어를 표현하는 임베딩 벡터 차원 출력


100


In [9]:
import numpy as np                                      # 수치 계산용 numpy

embedding_dim = fasttext_model.vector_size              # FastText 임베딩 벡터 차원
embedding_matrix = np.zeros((vocab_size, embedding_dim))# 단어 사전 크기 × 임베딩 차원 행렬 생성

word_index = tokenizer.word_index                       # 전체 단어 → 인덱스 사전 (예: 38,000개)
word_index = {word: index                               # vocab_size 이내 단어만 필터링
              for word, index in word_index.items()
              if index < vocab_size}
print(len(word_index))                                  # 실제 사용할 단어 수 확인 (예: 10,000)

for word, index in word_index.items():                  # 단어 사전 순회
    if word in fasttext_model.wv:                       # FastText 모델에 단어가 있으면
        embedding_matrix[index] = fasttext_model.wv[word]  # 해당 단어 임베딩 벡터 복사


9999


In [10]:
# 모델 생성
import torch.nn as nn                               # PyTorch 신경망 모듈

class LSTMClassifier2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embedding_matrix, hidden_size):
        super().__init__()                          # nn.Module 초기화
        # embedding → lstm → dense 구조

        self.embedding = nn.Embedding(              # 단어 인덱스를 임베딩 벡터로 변환
            vocab_size,                             # 단어 사전 크기
            embedding_dim,                          # 임베딩 벡터 차원
            padding_idx=0                           # PAD 토큰은 학습 영향 없음
        )
        self.embedding.weight.data.copy_(            # 사전학습된 FastText 임베딩 가중치 복사
            torch.from_numpy(embedding_matrix)
        )
        self.embedding.weight.requires_grad = True   # 임베딩을 미세조정(fine-tuning) 허용

        self.lstm = nn.LSTM(                        # 문맥 정보를 학습하는 LSTM
            embedding_dim,                          # 입력 차원 (임베딩 크기)
            hidden_size,                            # 은닉 상태 차원
            batch_first=True                        # (batch, seq, feature) 형태 사용
        )
        self.fc = nn.Linear(                        # 최종 분류용 선형 레이어
            hidden_size,                            # LSTM 은닉 상태 차원
            3                                       # 클래스 개수 (3개 뉴스그룹)
        )

    def forward(self, x):
        x = self.embedding(x)                       # (batch, seq) → (batch, seq, embed)
        _, (h, c) = self.lstm(x)                    # LSTM 통과 (h: 마지막 은닉 상태)
        out = self.fc(h[-1])                        # 마지막 타임스텝 은닉 상태로 분류
        return out                                  # (batch, 3) 로짓 출력


In [11]:
# 모델 학습
import torch.optim as optim                                  # 옵티마이저 모듈

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # cuda 가능하면 GPU 사용
embedding_dim = 100                                          # 임베딩 벡터 차원
hidden_size = 128                                            # LSTM 은닉 상태 차원

model = LSTMClassifier2(vocab_size, embedding_dim, embedding_matrix, hidden_size).to(device)  # 모델 생성 후 GPU 이동
criterion = nn.CrossEntropyLoss()                            # 다중 클래스 분류용 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.0001)        # Adam 옵티마이저 설정 (학습률 1e-4)

# 학습 루프 기록용
train_losses, train_accs = [], []                            # 학습 손실/정확도 기록
val_losses, val_accs = [], []                                # 검증 손실/정확도 기록

epochs = 100                                                 # 전체 학습 에폭 수
for epoch in range(epochs):

    # 학습
    model.train()                                            # 학습 모드 전환
    train_loss, train_correct, train_total = 0, 0, 0          # 에폭 누적 변수 초기화

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 배치를 GPU로 이동
        optimizer.zero_grad()                                # 이전 gradient 초기화
        output = model(X_batch)                              # 순전파 (클래스 로짓 출력)
        loss = criterion(output, y_batch)                    # 손실 계산
        loss.backward()                                      # 역전파
        optimizer.step()                                     # 파라미터 업데이트

        train_loss += loss.detach().cpu().item()             # 배치 손실 누적
        pred = output.argmax(dim=1)                          # 예측 클래스(최대 로짓 인덱스)
        train_correct += (pred == y_batch).sum().detach().cpu().item()  # 정답 개수 누적
        train_total += len(y_batch)                          # 전체 샘플 수 누적

    train_loss /= len(train_loader)                          # 에폭 평균 학습 손실
    train_acc = train_correct / train_total                  # 에폭 학습 정확도
    train_losses.append(train_loss)                          # 학습 손실 기록
    train_accs.append(train_acc)                             # 학습 정확도 기록

    # 검증
    model.eval()                                             # 평가 모드 전환
    val_loss, val_correct, val_total = 0, 0, 0                # 검증 누적 변수 초기화

    with torch.no_grad():                                    # gradient 계산 비활성화
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 배치를 GPU로 이동
            output = model(X_batch)                           # 검증 순전파
            loss = criterion(output, y_batch)                 # 검증 손실 계산

            val_loss += loss.detach().cpu().item()            # 검증 손실 누적
            pred = output.argmax(dim=1)                       # 예측 클래스
            val_correct += (pred == y_batch).sum().detach().cpu().item()  # 정답 누적
            val_total += len(y_batch)                         # 샘플 수 누적

        val_loss /= len(val_loader)                           # 에폭 평균 검증 손실
        val_acc = val_correct / val_total                     # 에폭 검증 정확도
        val_losses.append(val_loss)                           # 검증 손실 기록
        val_accs.append(val_acc)                              # 검증 정확도 기록

    # 출력 (train_loss, val_loss)
    print(f'Epoch {epoch + 1}/{epochs}: '                     # 현재 에폭 정보 출력
          f'Train Loss {train_loss:.4f}, '
          f'Train Acc {train_acc:.4f}, '
          f'Val Loss {val_loss:.4f}, '
          f'Val Acc {val_acc:.4f}, ')


Epoch 1/100: Train Loss 1.0995, Train Acc 0.3254, Val Loss 1.0942, Val Acc 0.3679, 
Epoch 2/100: Train Loss 1.0955, Train Acc 0.3561, Val Loss 1.0909, Val Acc 0.4123, 
Epoch 3/100: Train Loss 1.0922, Train Acc 0.4333, Val Loss 1.0881, Val Acc 0.4334, 
Epoch 4/100: Train Loss 1.0881, Train Acc 0.4725, Val Loss 1.0841, Val Acc 0.4101, 
Epoch 5/100: Train Loss 1.0823, Train Acc 0.4963, Val Loss 1.0781, Val Acc 0.4482, 
Epoch 6/100: Train Loss 1.0711, Train Acc 0.5058, Val Loss 1.0623, Val Acc 0.5074, 
Epoch 7/100: Train Loss 1.0039, Train Acc 0.5556, Val Loss 0.9198, Val Acc 0.5603, 
Epoch 8/100: Train Loss 0.9900, Train Acc 0.4921, Val Loss 0.9496, Val Acc 0.5370, 
Epoch 9/100: Train Loss 0.8955, Train Acc 0.6233, Val Loss 0.8860, Val Acc 0.6110, 
Epoch 10/100: Train Loss 0.8439, Train Acc 0.6116, Val Loss 0.8962, Val Acc 0.5793, 
Epoch 11/100: Train Loss 0.8038, Train Acc 0.6577, Val Loss 0.8250, Val Acc 0.6512, 
Epoch 12/100: Train Loss 0.7747, Train Acc 0.6757, Val Loss 0.7869, Val Ac

In [12]:
# 모델 평가
# - 정답 라벨과 모델 예측값을 사용해 classification_report 생성
from sklearn.metrics import classification_report    # 분류 성능 리포트 함수

model.eval()                                         # 모델을 평가 모드로 전환
all_preds, all_labels = [], []                       # 전체 예측값/정답 저장용 리스트

with torch.no_grad():                                # gradient 계산 비활성화
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 배치를 GPU로 이동
        output = model(X_batch)                      # 모델 예측값(로짓) 계산
        loss = criterion(output, y_batch)            # 테스트 손실 계산(로그/확인용)
        pred = output.argmax(dim=1)                  # 가장 큰 로짓을 갖는 클래스 선택

        all_preds.extend(pred.detach().cpu().numpy())    # 예측 라벨을 CPU numpy로 저장
        all_labels.extend(y_batch.detach().cpu().numpy())# 실제 라벨을 CPU numpy로 저장

print(
    classification_report(                           # 클래스별 precision/recall/f1 출력
        all_labels,                                  # 실제 라벨
        all_preds,                                   # 예측 라벨
        target_names=newsgroups.target_names          # 클래스 이름 매핑
    )
)


                    precision    recall  f1-score   support

     comp.graphics       0.93      0.93      0.93       202
rec.sport.baseball       0.95      0.95      0.95       202
         sci.space       0.93      0.93      0.93       187

          accuracy                           0.94       591
         macro avg       0.94      0.94      0.94       591
      weighted avg       0.94      0.94      0.94       591

